Get current SPY constituents, create dataset of last 5 years convert yfiance price to returns and convert to my dataset format and test on models rf and transformer

In [3]:
import pandas as pd
import vectorbt as vbt
import json

In [4]:
stocks=['AAPL','MSFT','AMZN','GOOG','IBM']
start = '2015-01-01 UTC'  # crypto is in UTC

end = '2024-01-01 UTC'

In [5]:
# file_path = '../data/grouped_companies.json'

# # Open the file and load its content into a Python dictionary
# with open(file_path, 'r') as file:
#     stocks_by_date = json.load(file)

In [6]:
data = []
for stock in stocks:
    df = vbt.YFData.download(stock, start=start, end=end).get('Close')
    df.name = stock
    data.append(df)

# Combining all data into a single DataFrame
prices = pd.concat(data, axis=1)



In [7]:
prices

,AAPL,MSFT,AMZN,GOOG,IBM
Date,,,,,
2015-01-02 05:00:00+00:00,24.466448,40.452728,15.426000,26.168653,104.297104
2015-01-05 05:00:00+00:00,23.777187,40.080742,15.109500,25.623152,102.655991
2015-01-06 05:00:00+00:00,23.779425,39.492458,14.764500,25.029282,100.442093
2015-01-07 05:00:00+00:00,24.112862,39.994228,14.921000,24.986401,99.785645
2015-01-08 05:00:00+00:00,25.039341,41.170776,15.023000,25.065184,101.954498
...,...,...,...,...,...
2023-12-22 05:00:00+00:00,193.600006,374.579987,153.419998,142.720001,162.139999
2023-12-26 05:00:00+00:00,193.050003,374.660004,153.410004,142.820007,163.210007
2023-12-27 05:00:00+00:00,193.149994,374.070007,153.339996,141.440002,163.460007


In [10]:
daily_returns = prices.pct_change()

In [11]:
daily_returns

,AAPL,MSFT,AMZN,GOOG,IBM
Date,,,,,
2015-01-02 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2015-01-05 05:00:00+00:00,-0.028172,-0.009196,-0.020517,-0.020846,-0.015735
2015-01-06 05:00:00+00:00,0.000094,-0.014677,-0.022833,-0.023177,-0.021566
2015-01-07 05:00:00+00:00,0.014022,0.012705,0.010600,-0.001713,-0.006536
2015-01-08 05:00:00+00:00,0.038423,0.029418,0.006836,0.003153,0.021735
...,...,...,...,...,...
2023-12-22 05:00:00+00:00,-0.005547,0.002784,-0.002730,0.006488,0.008459
2023-12-26 05:00:00+00:00,-0.002841,0.000214,-0.000065,0.000701,0.006599
2023-12-27 05:00:00+00:00,0.000518,-0.001575,-0.000456,-0.009663,0.001532


In [12]:
#Remove the first date
daily_returns=daily_returns.iloc[1:]

In [13]:
daily_returns

,AAPL,MSFT,AMZN,GOOG,IBM
Date,,,,,
2015-01-05 05:00:00+00:00,-0.028172,-0.009196,-0.020517,-0.020846,-0.015735
2015-01-06 05:00:00+00:00,0.000094,-0.014677,-0.022833,-0.023177,-0.021566
2015-01-07 05:00:00+00:00,0.014022,0.012705,0.010600,-0.001713,-0.006536
2015-01-08 05:00:00+00:00,0.038423,0.029418,0.006836,0.003153,0.021735
2015-01-09 05:00:00+00:00,0.001072,-0.008405,-0.011749,-0.012951,0.004356
...,...,...,...,...,...
2023-12-22 05:00:00+00:00,-0.005547,0.002784,-0.002730,0.006488,0.008459
2023-12-26 05:00:00+00:00,-0.002841,0.000214,-0.000065,0.000701,0.006599
2023-12-27 05:00:00+00:00,0.000518,-0.001575,-0.000456,-0.009663,0.001532


In [15]:
long_format_returns = daily_returns.reset_index().melt(id_vars='Date', var_name='TICKER', value_name='RET')


In [23]:
long_format_returns = long_format_returns.rename(columns={'Date': 'date'})


In [24]:
long_format_returns

,date,TICKER,RET
0,2015-01-05 05:00:00+00:00,AAPL,-0.028172
1,2015-01-06 05:00:00+00:00,AAPL,0.000094
2,2015-01-07 05:00:00+00:00,AAPL,0.014022
3,2015-01-08 05:00:00+00:00,AAPL,0.038423
4,2015-01-09 05:00:00+00:00,AAPL,0.001072
...,...,...,...
11310,2023-12-22 05:00:00+00:00,IBM,0.008459
11311,2023-12-26 05:00:00+00:00,IBM,0.006599
11312,2023-12-27 05:00:00+00:00,IBM,0.001532
11313,2023-12-28 05:00:00+00:00,IBM,0.001774


In [25]:
long_format_returns.to_csv('../data/modern_stock_data.csv')

In [29]:
# Resetting the index to make 'Date' a column
df_long = prices.reset_index()

# Melting the DataFrame
df_long = df_long.melt(id_vars=['Date'], var_name='ticker', value_name='prc')

# You might want to sort the DataFrame for better organization
df_long = df_long.sort_values(by=['Date', 'ticker'])

df_long.rename(columns={'prc': 'PRICE'}, inplace=True)
df_long.rename(columns={'Date':'date'},inplace=True)
df_long.rename(columns={'ticker':'TICKER'},inplace=True)
df_long['date'] = df_long['date'].dt.tz_localize(None)

In [30]:
df_long.to_csv('../data/vectorbt_data.csv')

In [26]:
import pandas as pd
from data_helper_functions import *
# Assuming df_long is your DataFrame

# Parameters for study periods
window_size = 240
trade_size = 250
train_size = 750
forward_roll = 250
start_date = pd.to_datetime("2014-01-01")
end_date = pd.to_datetime("2019-12-31")
target_type = 'direction'
standardize = True
data_type = 'PRICE'
apply_wavelet_transform = False

# Creating study periods
study_periods = create_study_periods(df_long, window_size, trade_size, train_size, forward_roll, start_date, end_date, target_type, standardize, data_type, apply_wavelet_transform)

# Parameters for tensor creation
n_jobs = 6
sequence_length = 240

# Creating tensors
# Parameters for tensor creation
n_jobs = 6
sequence_length = 240

# Creating tensors
train_test_splits, task_types = create_tensors(study_periods, n_jobs, sequence_length)


 56%|█████▌    | 5/9 [00:00<00:00, 162.02it/s]

Reached the end of the dataset.



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s
[Parallel(n_jobs=6)]: Done   3 out of   5 | elapsed:    1.6s remaining:    1.1s
[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:    1.6s finished
